# 6 A climate component of a planner's decision problem

This notebook corresponds to section 6 in the paper.

To check the previous section:

[Section 5: Uncertainty aversion](sec5_UncertaintyAversion.ipynb)

In [1]:
import numpy as np
import pandas as pd
from source.model import solve_hjb_y, solve_hjb_y_jump
from source.utilities import find_nearest_value
from source.simulation import simulate_jump
import plotly.graph_objects as go

## 6.1 Parameters
We use the following values for the parameters listed below throughout:

| Parameters | values |
| :---:| :---|
|$\delta$ |  0.01 |
|$\eta$ | 0.032 | 
|$\bar y$ | 2 |
|$\gamma_1$ | 0.00017675 |
|$\gamma_2$ | 0.0044|
|$\gamma_3^1$| 0.0000|
|$\gamma_3^2$| 0.0394|
|$\gamma_3^3$| 0.7706|
|$\varsigma'$| [0.0022, 0]|



## 6.2 Computation

Choose a set of uncertainty parameters $(\xi_b, \xi_p, \xi_a)$ and proceed with the following two step computation. We explore 144 climate models and 3 damage functions, *i.e.* $L=144$ and $M=3$.

For analysis of jump misspecification and ambiguity aversion, we set uncertainty parameter for brownian misspecification $\xi_b$ to $\infty$, and explore the roles of $\xi_a$ and $\xi_p$.

Here we set baseline parameters to be $(\xi_a, \xi_p) = (0.01,5)$, which is the values in the paper.
Then, we explore different values of $\xi_p$.
Specifically, the set of $\xi_p$'s in the following example is $(2\times 5, 5, 5/2, 5/10, 5/20)$.
Feel free to perform your own experiments.

###  6.2.1 Step I:

Solve the following HJB for $\gamma_3^m \in \{\gamma_3^1, \gamma_3^2, \gamma_3^3\}$ on $y \in [0,4]$ to get the corresponding $\phi_m(y)$ with $m = 1,2,3$:

$$
\begin{aligned}
0 = \max_{\tilde e} \min_{\omega^a_\ell : \sum_{\ell=1}^L \omega^a_\ell = 1}  &- \delta \phi_m(y) +  \eta \log\tilde e \\
& + \frac{1}{2} \left(\frac{d^2 \phi_m}{dy^2} + \frac{ (\eta - 1)}{\delta} \left(\gamma_2 + \gamma_3\mathbb{I}\{y>\bar y\} \right) \right)(\tilde e)^2 |\varsigma|^2  \\
& - \frac{1}{2\xi_b} \left[ \frac{d\phi_m}{dy}    + \frac{(\eta -1)}{\delta}(\gamma_1 + \gamma_2 y + \gamma_3 (y-\bar y)\mathbb{I}\{y > \bar y\})\right]^2 \cdot |\varsigma|^2 (\tilde e)^2 \\
\\
& + \sum_{\ell=1}^{L} \omega_\ell^a \left(\frac{d\phi_m}{dy}+ \frac{(\eta -1)}{\delta}(\gamma_1 + \gamma_2 y + \gamma_3^m (y - \bar y)\mathbb{I}\{y > \bar y\} ) \right)\theta_\ell \tilde e   \\
\\
& + \xi_a \sum_i \omega^a_\ell(\log \omega^a_\ell - \log \pi^a_\ell)
\end{aligned}
$$

### 6.2.2 Step II:

Assume the baseline probabilities for damage functions are $(\cfrac13, \cfrac13, \cfrac13)$.

Given the above $\phi_1$, $\phi_2$ and $\phi_3$, solve the following HJB for $\phi(y)$ on $y \in [0, \bar y]$:

$$
\begin{aligned}
0 = \max_{\tilde e} \min_{\omega^a_\ell : \sum_{\ell=1}^L \omega^a_\ell = 1}  &- \delta \phi(y) +  \eta \log\tilde e \\
&+ \frac{1}{2} \left(\frac{d^2 \phi_m}{dy^2} + \frac{ (\eta - 1)}{\delta} \left(\gamma_2 + \gamma_3\mathbb{I}\{y>\bar y\} \right) \right)(\tilde e)^2 |\varsigma|^2 \\
\\
& - \frac{1}{2\xi_b} \left[ \frac{d\phi}{dy}    + \frac{(\eta -1)}{\delta}(\gamma_1 + \gamma_2 y + \gamma_3 (y-\bar y)\mathbb{I}\{y > \bar y\})\right]^2 \cdot |\varsigma|^2 (\tilde e)^2 \\
\\
&  + \sum_{\ell=1}^{L} \omega_\ell^a \left(\frac{d\phi}{dy}+ \frac{(\eta -1)}{\delta}(\gamma_1 + \gamma_2 y + \gamma_3^m (y - \bar y)\mathbb{I}\{y > \bar y\} ) \right)\theta_\ell \tilde e   \\
\\
 & + \xi_a \sum_{\ell=1}^L \omega^a_\ell(\log \omega^a_\ell - \log \pi^a_\ell)
\end{aligned}
$$

$$
s.t \quad \phi(\bar y) \approx - \xi_p \log \left( \sum_{m=1}^M {\pi^d_m} \exp ( - \frac{1}{\xi_p}\phi_m(\bar y) )\right)
$$

In [2]:
# Uncertainty parameters
ξ_w = 100_000
ξ_a = 1/100
ξ_p = 5

# Preference
η = .032
δ = .01

# Climate sensitivity
θ_list = pd.read_csv('data/model144.csv', header=None).to_numpy()[:, 0]/1000.
πc_o = np.ones_like(θ_list)/len(θ_list)

# Damage function
σ_y = 1.2*np.mean(θ_list)
y_bar = 2.
γ_1 = 1.7675/10000
γ_2 = .0022*2
γ_2p = np.array([0, .0197*2, .3853*2])
πd_o = np.array([1./3, 1./3, 1./3])

y_step = .02
y_grid_long = np.arange(0., 4., y_step)
y_grid_short = np.arange(0., y_bar+y_step, y_step)
n_bar = find_nearest_value(y_grid_long, y_bar) + 1

# Prepare ϕ conditional on low, high, extreme damage
model_res_list = []
for γ_2p_i in γ_2p:
    model_args = (η, δ, σ_y, y_bar, γ_1, γ_2, γ_2p_i, θ_list, πc_o, ξ_w, ξ_a) 
    model_res = solve_hjb_y(y_grid_long, model_args, v0=None, ϵ=1.,
                            tol=1e-8, max_iter=5_000, print_iteration=False)
    model_res_list.append(model_res)

ϕ_list = [res['v'] for res in model_res_list]

solution = dict()
ξ_p_list = [100_000, 2*ξ_p, ξ_p, ξ_p/5, ξ_p/10, ξ_p/20]
for ξ_p_i in ξ_p_list:
    certainty_equivalent = -ξ_p_i*np.log(np.average(np.exp(-1./ξ_p_i*np.array(ϕ_list)), axis=0, weights=πd_o))
    # Change grid from 0-4 to 0-2
    ϕ_i = np.array([temp[:n_bar] for temp in ϕ_list])

    # Compute ϕ with jump (impose boundary condition)
    model_args = (η, δ, σ_y, y_bar, γ_1, γ_2, γ_2p, θ_list, πc_o, ϕ_i, πd_o, ξ_w, ξ_p_i, ξ_a)
    model_res = solve_hjb_y_jump(y_grid_short, model_args, 
                             v0=np.average(ϕ_i, weights=πd_o, axis=0),
                             ϵ=1., tol=1e-8, max_iter=5_000, print_iteration=False)
    simulation_res = simulate_jump(model_res, θ_list)
    solution[ξ_p_i] = dict(
        model_res=model_res, 
        simulation_res=simulation_res, 
        certainty_equivalent=certainty_equivalent
    )

Converged. Total iteration: 5000;	 LHS Error: 3.312423817458665e-05;	 RHS Error 8.65694637807446e-05
Converged. Total iteration: 1507;	 LHS Error: 9.965619884866328e-09;	 RHS Error 0.00047002697358666107
Converged. Total iteration: 1621;	 LHS Error: 9.95228432998374e-09;	 RHS Error 0.002278804686049074
Converged. Total iteration: 358;	 LHS Error: 9.58990753474609e-09;	 RHS Error 0.0006109671772151562
Converged. Total iteration: 338;	 LHS Error: 9.945289036750182e-09;	 RHS Error 0.0007150034845826783
Converged. Total iteration: 443;	 LHS Error: 9.751046192718604e-09;	 RHS Error 0.0008702963234703057
Converged. Total iteration: 539;	 LHS Error: 9.939043366102851e-09;	 RHS Error 0.0015641518605310854
Converged. Total iteration: 5000;	 LHS Error: 0.00010600966965901293;	 RHS Error 0.0018759982298651168
Converged. Total iteration: 509;	 LHS Error: 9.944462142641441e-09;	 RHS Error 0.002056194762227792


In [6]:
# value functions
fig = go.Figure()
for step in ξ_p_list:
    for ϕ, color, name in zip(ϕ_list, ["darkgreen", 'darkorange', "red"], ["low", "high", "extreme"]):
        fig.add_trace(go.Scatter(x=y_grid_long[n_bar:], y=ϕ[n_bar:], 
                                 visible=False,
                                 line=dict(color=color),
                                 name=name+" damage"
                                )) 
    fig.add_trace(go.Scatter(x=y_grid_short, y=solution[step]["model_res"]["v"], 
                             visible=False, 
                             name="prior to jump", 
                             line=dict(color="navy")
                            ))
    fig.add_trace(go.Scatter(x=y_grid_long[n_bar:], y=solution[step]["certainty_equivalent"][n_bar:], 
                             visible=False,
                             name="after jump, certainty equivalent",
                            line=dict(color="navy", dash="dash")))

fig.add_vline(x=2) 

num_lines = len(ϕ_list)+2
for i in range(num_lines):
    fig.data[1*num_lines + i].visible = True

title_text = "Continuation\ value\ function"
steps = []
for i in range(len(ξ_p_list)):
    # Hide all traces
    label = r'ξ_p = {:.2f}'.format(ξ_p_list[i])
    step = dict(
        method ='update',  
        args = [{'visible': [False] * len(fig.data)},
               {"title": r"${} \\ ξ_a = 0.01,\quad ξ_p = {:.2f}$".format(title_text, ξ_p_list[i])}],
        label=label
    )
    # Enable the two traces we want to see
    for j in range(len(ϕ_list)+2):
        step['args'][0]["visible"][5*i+j] = True
    # Add step to step list
    steps.append(step)

sliders = [dict(
    active = 2,
    currentvalue={"prefix": 'Jump misspecification '},
    steps = steps,
    pad={"t": 70},
)]

fig.update_layout(
    sliders = sliders,
    font=dict(size=16),
#     template="none"
)
fig.update_xaxes(
    linecolor='black',
    range=[0,3],
    title_text="y"
)
fig.update_yaxes(
    range=[10,-10],
    title_text="ϕ(y)"
)

Continue the next section, checkout:

[Section 7: Sensitivity](sec7_Sensitivity.ipynb)